In [77]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

In [2]:
from selenium import webdriver
import os
from selenium.webdriver import chrome
from selenium.webdriver.common.by import By

from selenium.webdriver.common.action_chains import ActionChains

import requests

from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()

driver.get('https://www.datasciencecentral.com/articles/')

driver.implicitly_wait(15)

# Switch to iframe
frame = driver.find_elements(By.TAG_NAME, 'iframe') 
driver.switch_to.frame("sp_message_iframe_1100486")

# Full xpath
button = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[4]/button[3]')

actions = ActionChains(driver)
actions.click(button)

actions.perform()

time.sleep(3)
previous_height = driver.execute_script('return document.body.scrollHeight')

# Scrolling
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

    time.sleep(3)

    new_height = driver.execute_script('return document.body.scrollHeight')
    
    if new_height == previous_height:
        break
    
    previous_height = new_height

links = []

soup=BeautifulSoup(driver.page_source)

link = soup.find_all(class_="nv-post-thumbnail-wrap img-wrap")

for l in link: 
    links.append(l.next['href'])

    
driver.close()

# print(links)

In [4]:
len(links)

2523

In [10]:
f = open("links.txt", "a")
for l in links:
    f.write(l + '\n')
f.close()

In [130]:
links[0]

'https://www.datasciencecentral.com/dsc-weekly-16-april-2024/'

In [133]:
from selenium.webdriver.chrome.options import Options

df = pd.DataFrame(columns = ['title', 'text', 'link'])

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

for link in links:


    driver.get(link)
    
    driver.implicitly_wait(3)
    
    try:
        # Switch to iframe
        frame = driver.find_elements(By.TAG_NAME, 'iframe') 
        driver.switch_to.frame("sp_message_iframe_1100486")
    
        # Full xpath
        button = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[4]/button[3]')
        
        actions = ActionChains(driver)
        actions.click(button)
        
        actions.perform()
    except: 
        pass
    
    soup=BeautifulSoup(driver.page_source)
    
    passage = ''

    if soup.find(class_ = 'nv-content-wrap entry-content') != None:
        for p in soup.find(class_ = 'nv-content-wrap entry-content').children:
            if p.string != None:
                passage += p.string
            else:
                try:
                    passage += p.text
                except:
                    pass

                         
    
    df = pd.concat([df, pd.DataFrame([{'title': link.split('/')[3], 'text': passage, 'link': link}])]) 
    
driver.close()

df.to_csv('articles.csv', index=False)

In [135]:
articles = pd.read_csv('articles.csv')
articles

,title,text,link
0,dsc-weekly-16-april-2024,\n\nAnnouncements\n\nIn today’s constantly evo...,https://www.datasciencecentral.com/dsc-weekly-...
1,using-window-functions-for-advanced-data-analysis,\nWindow functions are an advanced feature of ...,https://www.datasciencecentral.com/using-windo...
2,get-ready-for-future-innovations-with-large-la...,"\nNowadays, almost all businesses use generati...",https://www.datasciencecentral.com/get-ready-f...
3,5-mistakes-to-avoid-in-cmmc-compliance,\nThink of a battlefield — not filled with sol...,https://www.datasciencecentral.com/5-mistakes-...
4,building-reliable-and-efficient-etl-pipelines-...,\nData is crucial for your business—it helps w...,https://www.datasciencecentral.com/building-re...
...,...,...,...
2518,digital-strategy-series-part-i-creating-a-data...,"\nOh, the strategy pundits hate me! It’s not b...",https://www.datasciencecentral.com/digital-str...
2519,statistics-used-in-data-science-a-dictionary-i...,\nNaming conventions are often quite different...,https://www.datasciencecentral.com/statistics-...
2520,ai-superpowers-for-people-and-jobs,"\n \nIntroduction\nIn this post, I argue that\...",https://www.datasciencecentral.com/ai-superpow...
2521,how-anonymous-is-anonymous-data,\nThis post discusses what actually makes data...,https://www.datasciencecentral.com/how-anonymo...
